In [1]:
# Dependencies
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import sqlalchemy.ext 
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import create_engine, func
from sqlalchemy.orm import Session

In [2]:
# Name of the CSV file
file = Path('WineQualityData.csv')

In [3]:
# Read the CSV in pandas
df = pd.read_csv(file, encoding="ISO-8859-1")

In [4]:
# Preview of the DataFrame
df.head()

,ï»¿INDEX,TARGET,FixedAcidity,VolatileAcidity,CitricAcid,ResidualSugar,Chlorides,FreeSulfurDioxide,TotalSulfurDioxide,Density,pH,Sulphates,Alcohol,LabelAppeal,AcidIndex,STARS
0,1,3,3.2,1.160,-0.98,54.2,-0.567,NaN,268.0,0.99280,3.33,-0.59,9.9,0,8,2.0
1,2,3,4.5,0.160,-0.81,26.1,-0.425,15.0,-327.0,1.02792,3.38,0.70,NaN,-1,7,3.0
2,4,5,7.1,2.640,-0.88,14.8,0.037,214.0,142.0,0.99518,3.12,0.48,22.0,-1,8,3.0
3,5,3,5.7,0.385,0.04,18.8,-0.425,22.0,115.0,0.99640,2.24,1.83,6.2,-1,6,1.0
4,6,4,8.0,0.330,-1.26,9.4,NaN,-167.0,108.0,0.99457,3.12,1.77,13.7,0,9,2.0


In [5]:
# Delete extraneous column
del df['ï»¿INDEX']
df.head()

,TARGET,FixedAcidity,VolatileAcidity,CitricAcid,ResidualSugar,Chlorides,FreeSulfurDioxide,TotalSulfurDioxide,Density,pH,Sulphates,Alcohol,LabelAppeal,AcidIndex,STARS
0,3,3.2,1.160,-0.98,54.2,-0.567,NaN,268.0,0.99280,3.33,-0.59,9.9,0,8,2.0
1,3,4.5,0.160,-0.81,26.1,-0.425,15.0,-327.0,1.02792,3.38,0.70,NaN,-1,7,3.0
2,5,7.1,2.640,-0.88,14.8,0.037,214.0,142.0,0.99518,3.12,0.48,22.0,-1,8,3.0
3,3,5.7,0.385,0.04,18.8,-0.425,22.0,115.0,0.99640,2.24,1.83,6.2,-1,6,1.0
4,4,8.0,0.330,-1.26,9.4,NaN,-167.0,108.0,0.99457,3.12,1.77,13.7,0,9,2.0


In [6]:
# Identify incomplete rows
df.count()

TARGET                12795
FixedAcidity          12795
VolatileAcidity       12795
CitricAcid            12795
ResidualSugar         12179
Chlorides             12157
FreeSulfurDioxide     12148
TotalSulfurDioxide    12113
Density               12795
pH                    12400
Sulphates             11585
Alcohol               12142
LabelAppeal           12795
AcidIndex             12795
STARS                  9436
dtype: int64

In [7]:
# Drop all rows with missing information
df = df.dropna(how='any')

In [8]:
# Verify dropped rows
df.count()

TARGET                6436
FixedAcidity          6436
VolatileAcidity       6436
CitricAcid            6436
ResidualSugar         6436
Chlorides             6436
FreeSulfurDioxide     6436
TotalSulfurDioxide    6436
Density               6436
pH                    6436
Sulphates             6436
Alcohol               6436
LabelAppeal           6436
AcidIndex             6436
STARS                 6436
dtype: int64

In [9]:
# Verify data types.
df.dtypes

TARGET                  int64
FixedAcidity          float64
VolatileAcidity       float64
CitricAcid            float64
ResidualSugar         float64
Chlorides             float64
FreeSulfurDioxide     float64
TotalSulfurDioxide    float64
Density               float64
pH                    float64
Sulphates             float64
Alcohol               float64
LabelAppeal             int64
AcidIndex               int64
STARS                 float64
dtype: object

In [10]:
# Display an overview of the Target column
df['TARGET'].value_counts()

TARGET
4    2010
3    1469
5    1290
2     531
6     482
0     475
7      87
1      83
8       9
Name: count, dtype: int64

In [11]:
# Display an overview of the Stars column
df['STARS'].value_counts()

STARS
2.0    2450
1.0    2065
3.0    1527
4.0     394
Name: count, dtype: int64

In [12]:
# Display a statistical overview
df.describe()

,TARGET,FixedAcidity,VolatileAcidity,CitricAcid,ResidualSugar,Chlorides,FreeSulfurDioxide,TotalSulfurDioxide,Density,pH,Sulphates,Alcohol,LabelAppeal,AcidIndex,STARS
count,6436.000000,6436.000000,6436.000000,6436.000000,6436.000000,6436.000000,6436.000000,6436.000000,6436.000000,6436.000000,6436.000000,6436.000000,6436.000000,6436.000000,6436.000000
mean,3.669204,6.874767,0.301774,0.316425,5.540064,0.047968,32.448415,124.916952,0.993701,3.195207,0.505521,10.560168,0.044748,7.649938,2.038844
std,1.546017,6.281797,0.780066,0.860626,33.656724,0.316657,148.303784,228.930736,0.026508,0.678912,0.924748,3.715610,0.871291,1.191547,0.895321
min,0.000000,-18.000000,-2.750000,-3.160000,-127.800000,-1.171000,-555.000000,-793.000000,0.888090,0.480000,-3.130000,-4.500000,-2.000000,4.000000,1.000000
25%,3.000000,5.000000,0.120000,0.057500,-1.625000,-0.041250,3.000000,37.000000,0.986615,2.957500,0.260000,9.000000,-1.000000,7.000000,1.000000
50%,4.000000,6.800000,0.275000,0.310000,4.500000,0.044000,32.000000,126.000000,0.994050,3.190000,0.490000,10.500000,0.000000,7.000000,2.000000
75%,5.000000,9.025000,0.610000,0.580000,15.600000,0.133250,72.000000,209.000000,1.000205,3.460000,0.820000,12.400000,1.000000,8.000000,3.000000
max,8.000000,32.500000,3.680000,3.770000,140.650000,1.270000,622.000000,1057.000000,1.099240,5.940000,4.110000,26.500000,2.000000,17.000000,4.000000


In [13]:
# Save the DataFrame to a CSV file. 
df.to_csv("Cleaned_WineQualityData.csv", index=False)

In [12]:
#Read data and create dataframe

cleanedwine_df = pd.read_csv(r"C:\Users\sym0002\EdX_Bootcamp\Machine-Learning-Project\Data\Cleaned_WineQualityData.csv")

# Create a SQLite database engine
engine = create_engine('sqlite:///cleanedwine_df.sqlite')

# reflect an existing database into a new model
Base=automap_base()

# reflect the tables
Base.prepare(autoload_with=engine)

# Write DataFrame to SQLite database
cleanedwine_df.to_sql('cleaned_wine', con=engine, if_exists='replace', index=False)


In [8]:
# Query the database
query = "SELECT * FROM cleaned_wine"

# Execute the query and load the results into a DataFrame
result_df = pd.read_sql(query, engine)

# Display the DataFrame
print(result_df)

      TARGET  FixedAcidity  VolatileAcidity  CitricAcid  ResidualSugar  \
0          5           7.1            2.640       -0.88          14.80   
1          3           5.7            0.385        0.04          18.80   
2          6           5.5           -0.220        0.39           1.80   
3          0         -17.2            0.520        0.15         -33.80   
4          3           6.0            0.330       -1.06           3.00   
...      ...           ...              ...         ...            ...   
6431       3          -4.1            0.550        0.31          13.55   
6432       3           7.5            0.180        0.34           1.30   
6433       0           8.2            0.390        0.17           2.30   
6434       4           8.4            0.300        0.38          37.10   
6435       4           4.4            0.220        0.29           1.90   

      Chlorides  FreeSulfurDioxide  TotalSulfurDioxide  Density    pH  \
0         0.037              214.0    

In [9]:
cleanedwine_df.head()

,TARGET,FixedAcidity,VolatileAcidity,CitricAcid,ResidualSugar,Chlorides,FreeSulfurDioxide,TotalSulfurDioxide,Density,pH,Sulphates,Alcohol,LabelAppeal,AcidIndex,STARS
0,5,7.1,2.640,-0.88,14.8,0.037,214.0,142.0,0.99518,3.12,0.48,22.0,-1,8,3.0
1,3,5.7,0.385,0.04,18.8,-0.425,22.0,115.0,0.99640,2.24,1.83,6.2,-1,6,1.0
2,6,5.5,-0.220,0.39,1.8,-0.277,62.0,180.0,0.94724,3.09,0.75,12.6,0,8,4.0
3,0,-17.2,0.520,0.15,-33.8,-0.022,551.0,65.0,0.99340,4.31,0.56,13.1,1,5,1.0
4,3,6.0,0.330,-1.06,3.0,0.518,5.0,378.0,0.96643,3.55,-0.86,3.9,1,7,2.0
